# 00 — Data Intake and Validation (Core)

Decision Supported:
- D-PI-1 Provider Review Entry (PI)

Decision Actor:
- PI

Decision Owner:
Investigations Lead

Why This Notebook Exists:
To verify that the CMS Part B dataset is structurally capable of supporting provider-level anomaly detection and peer grouping without introducing artifacts or bias.

What Decision Changes Because Of This Work:
If the dataset fails validation, provider review prioritization cannot be evidence-driven and must be deferred or redesigned.

What This Notebook Does NOT Do:
- Does not flag providers
- Does not run anomaly models
- Does not infer behavior
- Does not produce operational recommendations

Primary Uncertainty Addressed:
- Data fitness: can the dataset support peer grouping and multi-signal anomaly detection without artifacts?

Data Dependencies:
- /data/raw/MUP_PHY_R25_P05_D23_Prov_Svc.csv

Outputs Produced (this notebook only):
- schema snapshot (columns, dtypes)
- row counts and basic completeness
- suppression/volume considerations (as detectable)
- interim parquet dataset in /data/interim/
- data quality log entry for evidence_log.md



In [1]:
# CELL-ID: DPI1-00-INGEST-01
# PURPOSE: Verify raw dataset exists and report size
# INPUTS: C:\Users\billm\Projects\Medicare\medicare-program-integrity\data\raw\MUP_PHY_R25_P05_D23_Prov_Svc.csv
# OUTPUTS: console confirmation + file size
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

# Hard anchor to project root (local environment)
PROJECT_ROOT = Path(r"C:\Users\billm\Projects\Medicare\medicare-program-integrity")

RAW_PATH = PROJECT_ROOT / "data" / "raw" / "MUP_PHY_R25_P05_V20_D23_Prov_Svc.csv"
INTERIM_DIR = PROJECT_ROOT / "data" / "interim"

assert RAW_PATH.exists(), f"Missing raw file: {RAW_PATH}"

file_size_gb = RAW_PATH.stat().st_size / (1024**3)

print(f"Raw file found: {RAW_PATH.name}")
print(f"Size (GB): {file_size_gb:.2f}")
print(f"Project root: {PROJECT_ROOT}")



Raw file found: MUP_PHY_R25_P05_V20_D23_Prov_Svc.csv
Size (GB): 2.85
Project root: C:\Users\billm\Projects\Medicare\medicare-program-integrity


In [2]:
# CELL-ID: DPI1-00-SCHEMA-01
# PURPOSE: Load a controlled sample to inspect columns and initial data types
# INPUTS: RAW_PATH (Part B PUF provider-service CSV)
# OUTPUTS: column list, sample rows, inferred pandas dtypes
# DECISION-LINK: D-PI-1 Provider Review Entry

import pandas as pd

SAMPLE_ROWS = 200_000  # schema discovery without stressing memory

sample = pd.read_csv(
    RAW_PATH,
    nrows=SAMPLE_ROWS,
    low_memory=False
)

print(f"Sample rows loaded: {len(sample):,}")
print(f"Column count: {len(sample.columns)}")
print("Columns:")
for c in sample.columns:
    print(" -", c)

display(sample.head(3))
display(sample.dtypes.to_frame("dtype"))


Sample rows loaded: 200,000
Column count: 28
Columns:
 - Rndrng_NPI
 - Rndrng_Prvdr_Last_Org_Name
 - Rndrng_Prvdr_First_Name
 - Rndrng_Prvdr_MI
 - Rndrng_Prvdr_Crdntls
 - Rndrng_Prvdr_Ent_Cd
 - Rndrng_Prvdr_St1
 - Rndrng_Prvdr_St2
 - Rndrng_Prvdr_City
 - Rndrng_Prvdr_State_Abrvtn
 - Rndrng_Prvdr_State_FIPS
 - Rndrng_Prvdr_Zip5
 - Rndrng_Prvdr_RUCA
 - Rndrng_Prvdr_RUCA_Desc
 - Rndrng_Prvdr_Cntry
 - Rndrng_Prvdr_Type
 - Rndrng_Prvdr_Mdcr_Prtcptg_Ind
 - HCPCS_Cd
 - HCPCS_Desc
 - HCPCS_Drug_Ind
 - Place_Of_Srvc
 - Tot_Benes
 - Tot_Srvcs
 - Tot_Bene_Day_Srvcs
 - Avg_Sbmtd_Chrg
 - Avg_Mdcr_Alowd_Amt
 - Avg_Mdcr_Pymt_Amt
 - Avg_Mdcr_Stdzd_Amt


,Rndrng_NPI,Rndrng_Prvdr_Last_Org_Name,Rndrng_Prvdr_First_Name,Rndrng_Prvdr_MI,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Ent_Cd,Rndrng_Prvdr_St1,Rndrng_Prvdr_St2,Rndrng_Prvdr_City,Rndrng_Prvdr_State_Abrvtn,...,HCPCS_Desc,HCPCS_Drug_Ind,Place_Of_Srvc,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,Avg_Mdcr_Pymt_Amt,Avg_Mdcr_Stdzd_Amt
0,1003000126,Enkeshafi,Ardalan,NaN,M.D.,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,...,Initial hospital care with straightforward or ...,N,F,12,12.0,12,250.226667,89.062500,60.312500,54.669167
1,1003000126,Enkeshafi,Ardalan,NaN,M.D.,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,...,Initial hospital care with straightforward or ...,N,F,22,22.0,22,318.581818,130.312727,99.380000,98.429545
2,1003000126,Enkeshafi,Ardalan,NaN,M.D.,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,...,Subsequent hospital care with straightforward ...,N,F,76,127.0,127,95.732283,54.820157,43.557323,38.748661


,dtype
Rndrng_NPI,int64
Rndrng_Prvdr_Last_Org_Name,object
Rndrng_Prvdr_First_Name,object
Rndrng_Prvdr_MI,object
Rndrng_Prvdr_Crdntls,object
Rndrng_Prvdr_Ent_Cd,object
Rndrng_Prvdr_St1,object
Rndrng_Prvdr_St2,object
Rndrng_Prvdr_City,object
Rndrng_Prvdr_State_Abrvtn,object


In [3]:
# CELL-ID: DPI1-00-SCHEMA-02
# PURPOSE: Define an explicit dtype plan for memory-safe chunk ingestion and parquet conversion
# INPUTS: sample.columns (schema) + RAW_PATH
# OUTPUTS: dtype_plan dict + sanity printout
# DECISION-LINK: D-PI-1 Provider Review Entry

# Explicit dtype plan (stable across chunks)
dtype_plan = {
    # Identifiers / descriptors (keep as string)
    "Rndrng_NPI": "string",
    "Rndrng_Prvdr_Last_Org_Name": "string",
    "Rndrng_Prvdr_First_Name": "string",
    "Rndrng_Prvdr_MI": "string",
    "Rndrng_Prvdr_Crdntls": "string",
    "Rndrng_Prvdr_Ent_Cd": "string",
    "Rndrng_Prvdr_St1": "string",
    "Rndrng_Prvdr_St2": "string",
    "Rndrng_Prvdr_City": "string",
    "Rndrng_Prvdr_State_Abrvtn": "string",
    "Rndrng_Prvdr_State_FIPS": "string",
    "Rndrng_Prvdr_Zip5": "string",
    "Rndrng_Prvdr_RUCA": "string",
    "Rndrng_Prvdr_RUCA_Desc": "string",
    "Rndrng_Prvdr_Cntry": "string",
    "Rndrng_Prvdr_Type": "string",
    "Rndrng_Prvdr_Mdcr_Prtcptg_Ind": "string",
    "HCPCS_Cd": "string",
    "HCPCS_Desc": "string",
    "HCPCS_Drug_Ind": "string",
    "Place_Of_Srvc": "string",

    # Counts / quantities
    "Tot_Benes": "Int64",
    "Tot_Srvcs": "Float64",            # can be non-integer in some CMS PUFs
    "Tot_Bene_Day_Srvcs": "Float64",

    # Averages (currency-like)
    "Avg_Sbmtd_Chrg": "Float64",
    "Avg_Mdcr_Alowd_Amt": "Float64",
    "Avg_Mdcr_Pymt_Amt": "Float64",
    "Avg_Mdcr_Stdzd_Amt": "Float64",
}

# Sanity: confirm we covered every column
missing_cols = [c for c in sample.columns if c not in dtype_plan]
extra_cols = [c for c in dtype_plan if c not in sample.columns]

print(f"Columns in sample: {len(sample.columns)}")
print(f"Columns in dtype_plan: {len(dtype_plan)}")
print("Missing from dtype_plan:", missing_cols)
print("Extra in dtype_plan:", extra_cols)


Columns in sample: 28
Columns in dtype_plan: 28
Missing from dtype_plan: []
Extra in dtype_plan: []


In [4]:
# CELL-ID: DPI1-00-QUALITY-01
# PURPOSE: Run basic data quality checks on the sample (missingness, ranges, key plausibility)
# INPUTS: sample (first 200k rows)
# OUTPUTS: missingness table, basic range checks, potential anomalies to annotate
# DECISION-LINK: D-PI-1 Provider Review Entry

import numpy as np
import pandas as pd

# Missingness %
missing_pct = (sample.isna().mean() * 100).sort_values(ascending=False).round(2)
display(missing_pct.to_frame("% missing (sample)"))

# Quick plausibility checks on key fields
checks = {}

# NPI length check (should be 10 when present)
npi_len = sample["Rndrng_NPI"].dropna().astype("string").str.len()
checks["NPI_len_not_10_pct"] = round((npi_len != 10).mean() * 100, 4) if len(npi_len) else None

# Tot_Benes should be >= 0
checks["Tot_Benes_negative_count"] = int((sample["Tot_Benes"] < 0).sum())

# Averages should be >= 0 (allow zeros)
for col in ["Avg_Sbmtd_Chrg", "Avg_Mdcr_Alowd_Amt", "Avg_Mdcr_Pymt_Amt", "Avg_Mdcr_Stdzd_Amt"]:
    checks[f"{col}_negative_count"] = int((sample[col] < 0).sum())

# Tot_Srvcs and Tot_Bene_Day_Srvcs should be >= 0
for col in ["Tot_Srvcs", "Tot_Bene_Day_Srvcs"]:
    checks[f"{col}_negative_count"] = int((sample[col] < 0).sum())

# Simple distribution snapshots (sample)
dist_cols = ["Tot_Benes", "Tot_Srvcs", "Avg_Mdcr_Pymt_Amt"]
display(sample[dist_cols].describe(include="all").T)

checks


,% missing (sample)
Rndrng_Prvdr_St2,75.22
Rndrng_Prvdr_MI,33.77
Rndrng_Prvdr_Crdntls,10.54
Rndrng_Prvdr_First_Name,4.89
Rndrng_Prvdr_RUCA_Desc,0.06
Rndrng_Prvdr_RUCA,0.06
Rndrng_NPI,0.00
Rndrng_Prvdr_Last_Org_Name,0.00
Rndrng_Prvdr_City,0.00
Rndrng_Prvdr_State_Abrvtn,0.00


,count,mean,std,min,25%,50%,75%,max
Tot_Benes,200000.0,74.231255,260.071849,11.0,17.000000,32.000000,72.000000,4.516900e+04
Tot_Srvcs,200000.0,283.209069,12057.958293,11.0,21.000000,43.000000,118.000000,5.208424e+06
Avg_Mdcr_Pymt_Amt,200000.0,84.034604,264.135577,0.0,21.140905,53.539819,92.595141,1.928284e+04


{'NPI_len_not_10_pct': np.float64(0.0),
 'Tot_Benes_negative_count': 0,
 'Avg_Sbmtd_Chrg_negative_count': 0,
 'Avg_Mdcr_Alowd_Amt_negative_count': 0,
 'Avg_Mdcr_Pymt_Amt_negative_count': 0,
 'Avg_Mdcr_Stdzd_Amt_negative_count': 0,
 'Tot_Srvcs_negative_count': 0,
 'Tot_Bene_Day_Srvcs_negative_count': 0}

In [5]:
# CELL-ID: DPI1-00-PARQUET-01
# PURPOSE: Convert large CSV into chunked parquet dataset for downstream analysis
# INPUTS: RAW_PATH, dtype_plan
# OUTPUTS: /data/interim/partb_provider_service.parquet/ (partitioned parquet files)
# DECISION-LINK: D-PI-1 Provider Review Entry
# WARNING: Heavy operation — may run 10–30 minutes depending on disk speed

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


INTERIM_PARQUET_DIR = INTERIM_DIR / "partb_provider_service.parquet"
INTERIM_PARQUET_DIR.mkdir(parents=True, exist_ok=True)

# Clean prior run if present
for f in INTERIM_PARQUET_DIR.glob("*.parquet"):
    f.unlink()

CHUNK_ROWS = 250_000

writer = None
total_rows = 0

for i, chunk in enumerate(pd.read_csv(
    RAW_PATH,
    chunksize=CHUNK_ROWS,
    dtype=dtype_plan,
    low_memory=False
)):
    table = pa.Table.from_pandas(chunk, preserve_index=False)

    out_file = INTERIM_PARQUET_DIR / f"part_{i:05d}.parquet"
    pq.write_table(table, out_file, compression="snappy")

    rows = len(chunk)
    total_rows += rows

    if i % 10 == 0:
        print(f"Chunks written: {i+1}, rows so far: {total_rows:,}")

print(f"Completed parquet write. Total rows written: {total_rows:,}")
print(f"Output location: {INTERIM_PARQUET_DIR}")


Chunks written: 1, rows so far: 250,000
Chunks written: 11, rows so far: 2,750,000
Chunks written: 21, rows so far: 5,250,000
Chunks written: 31, rows so far: 7,750,000
Completed parquet write. Total rows written: 9,660,647
Output location: C:\Users\billm\Projects\Medicare\medicare-program-integrity\data\interim\partb_provider_service.parquet


In [6]:
# CELL-ID: DPI1-00-VALIDATE-01
# PURPOSE: Validate parquet dataset is readable and schema is stable
# INPUTS: data/interim/partb_provider_service.parquet/*
# OUTPUTS: schema printout + 5-row sample
# DECISION-LINK: D-PI-1 Provider Review Entry

import pyarrow.dataset as ds

dataset = ds.dataset(INTERIM_PARQUET_DIR, format="parquet")
print(dataset.schema)

head_df = dataset.head(5).to_pandas()
display(head_df)


Rndrng_NPI: string
Rndrng_Prvdr_Last_Org_Name: string
Rndrng_Prvdr_First_Name: string
Rndrng_Prvdr_MI: string
Rndrng_Prvdr_Crdntls: string
Rndrng_Prvdr_Ent_Cd: string
Rndrng_Prvdr_St1: string
Rndrng_Prvdr_St2: string
Rndrng_Prvdr_City: string
Rndrng_Prvdr_State_Abrvtn: string
Rndrng_Prvdr_State_FIPS: string
Rndrng_Prvdr_Zip5: string
Rndrng_Prvdr_RUCA: string
Rndrng_Prvdr_RUCA_Desc: string
Rndrng_Prvdr_Cntry: string
Rndrng_Prvdr_Type: string
Rndrng_Prvdr_Mdcr_Prtcptg_Ind: string
HCPCS_Cd: string
HCPCS_Desc: string
HCPCS_Drug_Ind: string
Place_Of_Srvc: string
Tot_Benes: int64
Tot_Srvcs: double
Tot_Bene_Day_Srvcs: double
Avg_Sbmtd_Chrg: double
Avg_Mdcr_Alowd_Amt: double
Avg_Mdcr_Pymt_Amt: double
Avg_Mdcr_Stdzd_Amt: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 3852


,Rndrng_NPI,Rndrng_Prvdr_Last_Org_Name,Rndrng_Prvdr_First_Name,Rndrng_Prvdr_MI,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Ent_Cd,Rndrng_Prvdr_St1,Rndrng_Prvdr_St2,Rndrng_Prvdr_City,Rndrng_Prvdr_State_Abrvtn,...,HCPCS_Desc,HCPCS_Drug_Ind,Place_Of_Srvc,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,Avg_Mdcr_Pymt_Amt,Avg_Mdcr_Stdzd_Amt
0,1003000126,Enkeshafi,Ardalan,<NA>,M.D.,I,6410 Rockledge Dr Ste 304,<NA>,Bethesda,MD,...,Initial hospital care with straightforward or ...,N,F,12,12.0,12.0,250.226667,89.0625,60.3125,54.669167
1,1003000126,Enkeshafi,Ardalan,<NA>,M.D.,I,6410 Rockledge Dr Ste 304,<NA>,Bethesda,MD,...,Initial hospital care with straightforward or ...,N,F,22,22.0,22.0,318.581818,130.312727,99.38,98.429545
2,1003000126,Enkeshafi,Ardalan,<NA>,M.D.,I,6410 Rockledge Dr Ste 304,<NA>,Bethesda,MD,...,Subsequent hospital care with straightforward ...,N,F,76,127.0,127.0,95.732283,54.820157,43.557323,38.748661
3,1003000126,Enkeshafi,Ardalan,<NA>,M.D.,I,6410 Rockledge Dr Ste 304,<NA>,Bethesda,MD,...,Subsequent hospital care with moderate levelof...,N,F,180,341.0,341.0,194.441525,86.840762,69.086422,61.812258
4,1003000126,Enkeshafi,Ardalan,<NA>,M.D.,I,6410 Rockledge Dr Ste 304,<NA>,Bethesda,MD,...,Subsequent hospital care with moderate levelof...,N,F,53,79.0,79.0,251.454051,127.91519,100.943165,92.409114


In [7]:
# CELL-ID: DPI1-00-VALIDATE-02
# PURPOSE: Capture validation summary for evidence logging
# INPUTS: RAW_PATH, INTERIM_PARQUET_DIR, total_rows
# OUTPUTS: validation_summary dict (copy into evidence log)
# DECISION-LINK: D-PI-1 Provider Review Entry

validation_summary = {
    "raw_file": RAW_PATH.name,
    "raw_size_gb": round(RAW_PATH.stat().st_size / (1024**3), 2),
    "parquet_rows_written": int(total_rows),
    "interim_parquet_dir": str(INTERIM_PARQUET_DIR),
    "status": "OK",
    "notes": [
        "Chunked CSV ingestion completed without errors.",
        "Interim parquet dataset created for downstream peer grouping and signal engine.",
    ],
}
validation_summary


{'raw_file': 'MUP_PHY_R25_P05_V20_D23_Prov_Svc.csv',
 'raw_size_gb': 2.85,
 'parquet_rows_written': 9660647,
 'interim_parquet_dir': 'C:\\Users\\billm\\Projects\\Medicare\\medicare-program-integrity\\data\\interim\\partb_provider_service.parquet',
 'status': 'OK',
 'notes': ['Chunked CSV ingestion completed without errors.',
  'Interim parquet dataset created for downstream peer grouping and signal engine.']}

In [8]:
# CELL-ID: DPI1-00-EVIDENCE-01
# PURPOSE: Create/update D-PI-1 evidence log from executed results (no manual edits)
# INPUTS: PROJECT_ROOT, RAW_PATH, INTERIM_PARQUET_DIR, total_rows
# OUTPUTS: decisions/D-PI-1_provider_review/evidence_log.md
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

evidence_path = PROJECT_ROOT / "decisions" / "D-PI-1_provider_review" / "evidence_log.md"
evidence_path.parent.mkdir(parents=True, exist_ok=True)

header = (
    "| Evidence ID | Source | Notebook (Cell ID) | Result | Decision Relevance | Status |\n"
    "|---|---|---|---|---|---|\n"
)

evidence_id = "E-DPI1-00"
row = (
    f"| {evidence_id} | CMS Part B PUF Provider–Service (raw CSV → interim parquet) | "
    f"00_data_intake_and_validation.ipynb (DPI1-00-PARQUET-01) | "
    f"Interim parquet created; {int(total_rows):,} rows | "
    f"Enables peer grouping + signal engine for D-PI-1 | Accepted |\n"
)

# Read existing (if any) and upsert by Evidence ID
existing = evidence_path.read_text(encoding="utf-8") if evidence_path.exists() else ""
if not existing.strip():
    evidence_path.write_text(header + row, encoding="utf-8")
else:
    if header.strip() not in existing:
        existing = header + existing.lstrip()

    lines = existing.splitlines(keepends=True)
    # Remove any prior row with the same Evidence ID
    lines = [ln for ln in lines if not ln.startswith(f"| {evidence_id} |")]
    # Ensure file ends with newline
    if lines and not lines[-1].endswith("\n"):
        lines[-1] = lines[-1] + "\n"
    # Append updated row
    lines.append(row)

    evidence_path.write_text("".join(lines), encoding="utf-8")

print(f"Evidence log updated: {evidence_path}")


Evidence log updated: C:\Users\billm\Projects\Medicare\medicare-program-integrity\decisions\D-PI-1_provider_review\evidence_log.md


In [14]:
# CELL-ID: DPI1-00-EVIDENCE-02
# PURPOSE: Update evidence log entries E-DPI1-02 through E-DPI1-04
#          to reflect recalibrated MIN_BENES=50 run
# INPUTS: PROJECT_ROOT
# OUTPUTS: decisions/D-PI-1_provider_review/evidence_log.md (updated)
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\billm\Projects\Medicare\medicare-program-integrity")
evidence_path = PROJECT_ROOT / "decisions" / "D-PI-1_provider_review" / "evidence_log.md"

# Read existing log
existing = evidence_path.read_text(encoding="utf-8")

# Define updated rows (recalibrated MIN_BENES=50 run)
updates = {
    "E-DPI1-02": "| E-DPI1-02 | provider_signal_scores_v1.parquet | 02_signal_engine.ipynb (DPI1-02-OUTLIER-01) | Peer-adjusted Z-scores computed for all 8 signals; 970,848 providers after MIN_BENES=50 filter; 204,433 excluded | Enables multi-signal agreement tier assignment | Accepted |\n",
    "E-DPI1-03": "| E-DPI1-03 | provider_tiered_v1.parquet | 02_signal_engine.ipynb (DPI1-02-TIER-01) | Tier 1: 22,400 — Tier 2: 39,016 — Tier 3: 68,721 — No Flag: 840,711 — Total: 970,848 | Supports D-PI-1 review prioritization | Accepted |\n",
    "E-DPI1-04": "| E-DPI1-04 | provider_review_queue_v1.parquet | 02_signal_engine.ipynb (DPI1-02-RANK-01) | Tier 1 providers ranked by allowed dollars; 22,400 total; top entry IDTF $132M, 5 signals flagged | Actionable review queue for D-PI-1 post MIN_BENES recalibration | Accepted |\n",
}

# Replace stale rows line by line
lines = existing.splitlines(keepends=True)
new_lines = []
for line in lines:
    replaced = False
    for evidence_id, new_row in updates.items():
        if line.startswith(f"| {evidence_id} |"):
            new_lines.append(new_row)
            replaced = True
            break
    if not replaced:
        new_lines.append(line)

evidence_path.write_text("".join(new_lines), encoding="utf-8")
print(f"Evidence log updated: {evidence_path}")
print()
print("Updated entries:")
for eid in updates.keys():
    print(f"  {eid} — recalibrated to MIN_BENES=50 run")

Evidence log updated: C:\Users\billm\Projects\Medicare\medicare-program-integrity\decisions\D-PI-1_provider_review\evidence_log.md

Updated entries:
  E-DPI1-02 — recalibrated to MIN_BENES=50 run
  E-DPI1-03 — recalibrated to MIN_BENES=50 run
  E-DPI1-04 — recalibrated to MIN_BENES=50 run


In [9]:
# CELL-ID: DPI1-00-CHARTER-01
# PURPOSE: Write D-PI-1 decision charter README programmatically
# INPUTS: PROJECT_ROOT
# OUTPUTS: decisions/D-PI-1_provider_review/README.md
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\billm\Projects\Medicare\medicare-program-integrity")
charter_path = PROJECT_ROOT / "decisions" / "D-PI-1_provider_review" / "README.md"
charter_path.parent.mkdir(parents=True, exist_ok=True)

charter_content = """# Decision Charter — D-PI-1 Provider Review Entry

**Version:** 1.0
**Date:** February 2026
**Status:** Active

---

## Organizational Reference Notice

> Organizational roles and structures referenced in this document are based on the
> CMS Organizational Chart dated April 22, 2025. Structure is subject to change.
> Decision authority patterns described remain analytically applicable regardless
> of subsequent organizational changes. Data references are based on CMS Medicare
> Part B Public Use File, Reporting Year 2025, Data Year 2023 (RY25/D23).
> Readers should verify current organizational alignment independently.

---

## Decision

Which providers should enter the program integrity review queue based on
defensible, evidence-supported signals of anomalous utilization or billing
patterns relative to peer providers?

---

## Decision Owner

**Primary:**
Center for Program Integrity (CPI) — Analytics and Enforcement units
responsible for identifying and acting on provider-level program integrity risks.

**Supporting:**
Office of Enterprise Data and Analytics (OEDA) — Responsible for data
governance, analytical standards, and measurement methodology that informs
integrity signals.

**Secondary:**
Center for Medicare (CM) — Payment policy context and provider type
classification standards that define peer grouping logic.

---

## Decision Actor Tag

`PI` — Program Integrity

---

## Decision Owner Role

Investigations Lead / Program Integrity Analytics Manager

---

## Cadence

- Operational: Monthly review queue refresh
- Strategic: Quarterly threshold calibration review

---

## Operational Action

Select and prioritize providers (identified by NPI) for one of the following
actions based on evidence-supported anomaly signals:

- Tier 1 — Initiate focused medical review or audit
- Tier 2 — Place under active monitoring
- Tier 3 — Flag for passive surveillance
- No Flag — No action warranted at this time

---

## Decision Context

### Why This Decision Exists

CMS program integrity resources are finite. The Medicare Part B program covers
over one million rendering providers billing across thousands of procedure codes
nationally. Manual review of all providers is operationally impossible.

A structured, reproducible, evidence-based method is required to identify which
providers warrant review based on objective signals rather than reactive
complaint-driven selection.

Without this decision support, enforcement resources risk being:
- Misallocated to low-risk providers
- Delayed in reaching high-risk patterns
- Inconsistent across review cycles
- Indefensible under audit or legal scrutiny

### What Happens If Wrong

**False Positives (flagging legitimate providers):**
- Unnecessary administrative burden on providers
- Erosion of provider trust in CMS oversight
- Consumption of enforcement capacity on low-yield reviews
- Potential legal or reputational exposure

**False Negatives (missing high-risk providers):**
- Continued improper payment exposure
- Delayed identification of systemic billing patterns
- Reduced program integrity effectiveness
- Financial harm to the Medicare Trust Fund

### What Happens If Delayed

- Anomalous billing patterns persist undetected
- Payment exposure compounds over reporting cycles
- Systemic patterns become entrenched before intervention
- Enforcement credibility weakens

---

## Primary Uncertainties

| ID | Uncertainty | Decision Impact |
|---|---|---|
| U1 | What constitutes meaningful anomaly vs legitimate specialty variation? | Changes who gets flagged |
| U2 | How much variation is explained by patient mix vs provider behavior? | Affects false positive rate |
| U3 | Which signals persist across metrics vs appearing in only one? | Drives tier assignment confidence |
| U4 | What thresholds balance detection with operational review capacity? | Determines queue size |
| U5 | Are outliers artifacts of data suppression or real patterns? | Affects ranking reliability |

---

## Evidence Required

- Peer-adjusted utilization patterns (services per beneficiary)
- Payment intensity deviations (allowed and payment per beneficiary)
- Submitted-to-allowed charge ratios
- Code concentration signals (top HCPCS share)
- Allowed vs payment gap signals
- Multi-signal agreement across independent metrics
- Minimum volume reliability filters

---

## Data Dependencies

**Primary Dataset:**
CMS Medicare Part B Public Use File — Provider and Service Level
`MUP_PHY_R25_P05_V20_D23_Prov_Svc.csv`
Reporting Year 2025 / Data Year 2023

**Reference Documentation:**
- Data Dictionary: `MUP_PHY_RY25_20250312_DD_PRV_SVC_508.pdf`
- Technical Specifications: `MUP_PHY_RY25_20250312_Technical_Specifications_508.pdf`
- Methodology: `MUP_PHY_RY25_20250312_Methodology_508.pdf`

**Known Data Gaps:**
- No patient-level clinical or risk data
- No claims-level detail
- No intent or compliance indicators
- No real-time billing data (lagged reporting cycle)
- Patient complexity proxied via service intensity (Assumed — A)

---

## Analytical Requirements

- Peer grouping by Provider Type (primary) with State and RUCA
  as secondary diagnostic controls
- Outlier detection via peer-adjusted Z-scores across 8 independent signals
- Multi-signal agreement framework (Tier 1 requires >= 3 signals flagged)
- Minimum beneficiary volume filter (Tot_Benes >= 11)
- Prioritization by allowed dollars within each tier

---

## Output Tagging Standard

All outputs carry one of the following evidence tags:

| Tag | Meaning |
|---|---|
| O | Observed — directly in the data |
| D | Derived — mathematically constructed from observed data |
| I | Inferred — evidence-based pattern interpretation |
| A | Assumed — declared gap-filling for decision support |

**No output may assert provider intent, fraud, or wrongdoing.**
Language is restricted to: anomalous patterns, statistical deviation,
billing intensity signals, and peer-adjusted outliers.

---

## Outputs Produced

- `provider_review_queue_v1.parquet` — Ranked Tier 1 provider list (D)
- `provider_tiered_v1.parquet` — Full provider tier assignments (D)
- `provider_signal_scores_v1.parquet` — Peer-adjusted Z-scores (D)
- `signal_validation_v1.png` — Visual validation of signal distribution (D)
- Evidence log entries E-DPI1-00 through E-DPI1-04

---

## Decision Impact Pathway

Observed utilization (O)
  -> Derived peer-adjusted signals (D)
    -> Persistent multi-signal anomalies (I)
      -> Tiered review prioritization (D)
        -> Operational investigation queue
          -> Audit / Monitor / Education / No Action

---

## Notebook Dependencies

| Notebook | Role | Cell IDs |
|---|---|---|
| `00_data_intake_and_validation.ipynb` | Ingestion + parquet foundation | DPI1-00-* |
| `01_peer_grouping_design.ipynb` | Peer group definition + lock | DPI1-01-* |
| `02_signal_engine.ipynb` | Signal computation + tiering | DPI1-02-* |

---

## Governance Footnote

This charter reflects decision framing established February 2026.
CMS organizational references sourced from CMS Org Chart, April 22, 2025.
Data sourced from CMS Medicare Part B PUF, Data Year 2023.
All analytical outputs describe statistical patterns only.
No outputs constitute findings of fraud, waste, or abuse.
"""

charter_path.write_text(charter_content, encoding="utf-8")
print(f"Charter written: {charter_path}")

Charter written: C:\Users\billm\Projects\Medicare\medicare-program-integrity\decisions\D-PI-1_provider_review\README.md


In [10]:
# CELL-ID: DPI1-00-NOTEBOOKMAP-01
# PURPOSE: Write notebook_map.md for D-PI-1 programmatically
# INPUTS: PROJECT_ROOT
# OUTPUTS: decisions/D-PI-1_provider_review/notebook_map.md
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\billm\Projects\Medicare\medicare-program-integrity")
map_path = PROJECT_ROOT / "decisions" / "D-PI-1_provider_review" / "notebook_map.md"

map_content = """# Notebook Map — D-PI-1 Provider Review Entry

**Version:** 1.0
**Date:** February 2026

---

## Organizational Reference Notice

> Organizational roles and structures referenced in this document are based on the
> CMS Organizational Chart dated April 22, 2025. Structure is subject to change.
> Data references are based on CMS Medicare Part B PUF, RY25/D23.

---

## Purpose

This document maps each notebook to its role in supporting D-PI-1.
No notebook exists in this project without a decision justification.

---

## Core Notebooks (Shared Infrastructure)

| Notebook | Role | Key Cell IDs | Outputs |
|---|---|---|---|
| `00_data_intake_and_validation.ipynb` | Raw CSV ingestion, schema validation, parquet conversion | DPI1-00-INGEST-01 through DPI1-00-CHARTER-01 | `data/interim/partb_provider_service.parquet` |
| `01_peer_grouping_design.ipynb` | Peer group definition and lock (Provider Type primary) | DPI1-01-LOAD-01 through DPI1-01-LOCK-01 | `peer_group_contract` dict |
| `02_signal_engine.ipynb` | Signal computation, Z-scoring, tiering, review queue | DPI1-02-SIGNALS-01 through DPI1-02-EVIDENCE-01 | `provider_review_queue_v1.parquet` |

---

## Decision Notebooks (D-PI-1 Specific)

| Notebook | Role | Status |
|---|---|---|
| `D1_provider_review_queue.ipynb` | Frames Tier 1 queue as operational recommendation for PI actor | Pending |

---

## Dependency Order

Notebooks must be executed in this order per session:

1. `00_data_intake_and_validation.ipynb`
2. `01_peer_grouping_design.ipynb`
3. `02_signal_engine.ipynb`
4. `D1_provider_review_queue.ipynb` (decision notebook — pending)

---

## Governing Rule

A notebook is added to this map if and only if:

- A decision exists that requires it
- That decision has a blocking uncertainty
- That uncertainty requires unique evidence
- That evidence cannot be produced by an existing notebook

---

## Governance Footnote

CMS organizational references sourced from CMS Org Chart, April 22, 2025.
Data sourced from CMS Medicare Part B PUF, Data Year 2023.
"""

map_path.write_text(map_content, encoding="utf-8")
print(f"Notebook map written: {map_path}")

Notebook map written: C:\Users\billm\Projects\Medicare\medicare-program-integrity\decisions\D-PI-1_provider_review\notebook_map.md


In [11]:
# CELL-ID: DPI1-00-DECISIONLOG-01
# PURPOSE: Write decision_log.md for D-PI-1 programmatically
# INPUTS: PROJECT_ROOT
# OUTPUTS: decisions/D-PI-1_provider_review/decision_log.md
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\billm\Projects\Medicare\medicare-program-integrity")
log_path = PROJECT_ROOT / "decisions" / "D-PI-1_provider_review" / "decision_log.md"

log_content = """# Decision Log — D-PI-1 Provider Review Entry

**Version:** 1.0
**Date:** February 2026

---

## Organizational Reference Notice

> Organizational roles and structures referenced in this document are based on the
> CMS Organizational Chart dated April 22, 2025. Structure is subject to change.
> Data references are based on CMS Medicare Part B PUF, RY25/D23.

---

## Purpose

This log records analytical and architectural decisions made during the
development of D-PI-1 support infrastructure. It captures what was decided,
why, what evidence was used, and whether any decision was later reversed.

This is not a log of provider-level findings.
It is a log of analytical design decisions.

---

## Log Format

| ID | Date | Decision Made | Evidence Used | Confidence | Reversed? | Notes |
|---|---|---|---|---|---|---|

---

## Entries

| ID | Date | Decision Made | Evidence Used | Confidence | Reversed? | Notes |
|---|---|---|---|---|---|---|
| DL-DPI1-01 | 2026-02 | Primary peer grouping set to Provider Type only | DPI1-01-DIAG-01 through DIAG-03: Type-only p10=48, median=2584 vs fragmented results at Type x State | High | No | State-level grouping caused over-fragmentation; RUCA retained as secondary diagnostic control |
| DL-DPI1-02 | 2026-02 | Minimum volume filter set to Tot_Benes >= 11 | CMS PUF suppression rules suppress records below this threshold prior to publication | High | No | Filter confirmed zero exclusions in current dataset; retained for future data releases |
| DL-DPI1-03 | 2026-02 | Z-score threshold set to 2.0 for signal flagging | Standard statistical threshold for anomaly detection; calibration review scheduled quarterly | Medium | Pending review | Tier thresholds subject to operational capacity calibration in decision notebook |
| DL-DPI1-04 | 2026-02 | Tier 1 defined as >= 3 signals flagged | Multi-signal agreement reduces false positive risk; single-signal flags insufficient for review prioritization | High | No | Tier 2 = 2 signals, Tier 3 = 1 signal |
| DL-DPI1-05 | 2026-02 | Tier 1 prioritization ranked by allowed dollars | Highest exposure providers represent greatest financial risk to Medicare Trust Fund | High | No | Payment dollars used as secondary sort where allowed dollars are equal |
| DL-DPI1-06 | 2026-02 | 8 signals selected for v1 signal engine | Covers utilization, payment intensity, pricing behavior, and coding pattern dimensions independently | High | No | S1-S7 plus S5a/S5b; additional signals deferred to v2 pending decision notebook calibration |

---

## Reversal Protocol

If any decision above is reversed in a future session:

1. Update the Reversed? field to Yes
2. Add a new entry documenting the replacement decision
3. Update the evidence log with new supporting evidence
4. Update notebook_map.md if notebook dependencies change
5. Re-run affected notebooks in dependency order

---

## Governance Footnote

CMS organizational references sourced from CMS Org Chart, April 22, 2025.
Data sourced from CMS Medicare Part B PUF, Data Year 2023.
All decisions documented here reflect analytical design choices only.
No entries constitute findings of fraud, waste, or abuse.
"""

log_path.write_text(log_content, encoding="utf-8")
print(f"Decision log written: {log_path}")


Decision log written: C:\Users\billm\Projects\Medicare\medicare-program-integrity\decisions\D-PI-1_provider_review\decision_log.md


In [12]:
# CELL-ID: DPI1-00-DECISIONLOG-02
# PURPOSE: Update decision log with hardcoding annotation entries
# INPUTS: PROJECT_ROOT
# OUTPUTS: decisions/D-PI-1_provider_review/decision_log.md (appended)
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\billm\Projects\Medicare\medicare-program-integrity")
log_path = PROJECT_ROOT / "decisions" / "D-PI-1_provider_review" / "decision_log.md"

new_entries = [
    "| DL-DPI1-07 | 2026-02 | CORE_Z_THRESHOLD fixed at 2.0 in core signal engine | Shared infrastructure must produce statistically standard baseline used by all decision notebooks A-E; parameterized control lives in decision notebooks only | High | No | Documented in DPI1-02-TIER-01 hardcoding notice; change requires re-run of all downstream decision notebooks |\n",
    "| DL-DPI1-08 | 2026-02 | Tier boundary values (3,2,1) fixed in core signal engine | Core engine produces baseline tier structure only; operational threshold control (MIN_SIGNALS) lives exclusively in decision notebooks to prevent analytic bias leaking into shared infrastructure | High | No | Documented in DPI1-02-TIER-01 hardcoding notice; aligns with no-hardcoding standard by isolating fixed values with explicit rationale |\n",
]

existing = log_path.read_text(encoding="utf-8") if log_path.exists() else ""

with open(log_path, "a", encoding="utf-8") as f:
    for entry in new_entries:
        entry_id = entry.split("|")[1].strip()
        if entry_id not in existing:
            f.write(entry)

print(f"Decision log updated: {log_path}")

Decision log updated: C:\Users\billm\Projects\Medicare\medicare-program-integrity\decisions\D-PI-1_provider_review\decision_log.md


In [13]:
# CELL-ID: DPI1-00-DECISIONLOG-03
# PURPOSE: Record MIN_BENES recalibration decision in D-PI-1 decision log
# INPUTS: PROJECT_ROOT
# OUTPUTS: decisions/D-PI-1_provider_review/decision_log.md (appended)
# DECISION-LINK: D-PI-1 Provider Review Entry

from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\billm\Projects\Medicare\medicare-program-integrity")
log_path = PROJECT_ROOT / "decisions" / "D-PI-1_provider_review" / "decision_log.md"

new_entries = [
    "| DL-DPI1-09 | 2026-02 | EXTREME_Z_THRESHOLD fixed at 8.0 in D1-DPI1-DIAG-01 | Diagnostic boundary only; 4x standard flagging threshold (2.0); recalibrate if data composition changes materially | Medium | No | Named variable with hardcoding notice; not an operational threshold |\n",
    "| DL-DPI1-10 | 2026-02 | MIN_BENES recalibrated from 11 to 50 | Diagnostic D1-DPI1-DIAG-01 revealed providers with Tot_Benes 11-25 producing mathematically extreme Z-scores due to unstable ratios at low volume; peer group size confirmed not the cause | High | No | Raises minimum beneficiary volume filter; requires rerun of DPI1-02-OUTLIER-01 through D1-DPI1-VIZ-01 |\n",
]

existing = log_path.read_text(encoding="utf-8") if log_path.exists() else ""

with open(log_path, "a", encoding="utf-8") as f:
    for entry in new_entries:
        entry_id = entry.split("|")[1].strip()
        if entry_id not in existing:
            f.write(entry)

print(f"Decision log updated: {log_path}")

Decision log updated: C:\Users\billm\Projects\Medicare\medicare-program-integrity\decisions\D-PI-1_provider_review\decision_log.md
